In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", None)



# 1. Let's start with cleaning the data set named 'Business Interactions'

In [3]:
df_BI = pd.read_csv("/content/drive/MyDrive/Python Codes/data/raw/BusinessInteraction.csv", sep=';', on_bad_lines='warn')
df_BI.head()

,BusinessInteractionId,InstanceId,StepId,ActionCd,ActionUserId,StatusCd,EntityStatusCd,RelatedPartyId,Reason,StartTime,EndTime,AppActionCd,CreateUserId,CreateUserTime,UpdateUserId,UpdateUserTime,TargetUserId,TargetGroupId
0,10230767264,10003850078,1.000012e+12,SaveStep,573540,W,NEW,347878,NaN,2021-03-19 23:26:18.479,2021-03-19 23:26:18.479,NaN,573540,2021-03-19 23:26:18.588,NaN,NaN,0,0
1,10230767256,10003850078,NaN,WorkflowStarted,573540,W,NEW,573540,NaN,2021-03-19 23:26:02.260,2021-03-19 23:26:02.260,NaN,573540,2021-03-19 23:26:11.510,NaN,NaN,0,0
2,10230767260,10003850078,1.000012e+12,EmailSent,573540,W,NEW,347878,NaN,2021-03-19 23:26:14.104,2021-03-19 23:26:14.104,NaN,573540,2021-03-19 23:26:14.729,NaN,NaN,0,0
3,10232343851,10003850078,NaN,CancelInstance,4,W,NEW,4,AUTO DRAFT DELETE,2021-03-24 10:05:48.456,2021-03-24 10:05:48.456,HideBI,4,2021-03-24 10:05:52.972,NaN,NaN,0,0
4,10230767273,10003850078,1.000012e+12,SaveStep,573540,W,NEW,347878,NaN,2021-03-19 23:26:21.202,2021-03-19 23:26:21.202,NaN,573540,2021-03-19 23:26:21.390,NaN,NaN,0,0


In [4]:
#print(f"Number of rows and columns in df_BI: {df_BI.shape}")
print(f"Number of rows in df_BI: {df_BI.shape[0]}")
print(f"Number of columns in df_BI: {df_BI.shape[1]}")

Number of rows in df_BI: 1209091
Number of columns in df_BI: 18


In [5]:
df_BI.isnull().sum()

,0
BusinessInteractionId,0
InstanceId,0
StepId,47547
ActionCd,0
ActionUserId,0
StatusCd,0
EntityStatusCd,0
RelatedPartyId,0
Reason,1191669
StartTime,0


# What to do with NAs?
Reason, AppActionCd, UpdateUserTime and UpdateUserId are the columns where most of the cells are empty. Since the userIds and times are not that important and also the Reason column is almost empty, let's drop them. We keep StepID NA rows as they are NA because they cosrrespond to the initial Step (work flow starting step). We can replace the NAs on those cells in the next iteration.

In [6]:
columns_to_drop = ['Reason', 'AppActionCd', 'UpdateUserTime', 'UpdateUserId']
df_BI_cleaned = df_BI.drop(columns=columns_to_drop)

print("Original columns:", df_BI.columns.tolist())
print("\nColumns after dropping:", df_BI_cleaned.columns.tolist())
df_BI_cleaned.head()

Original columns: ['BusinessInteractionId', 'InstanceId', 'StepId', 'ActionCd', 'ActionUserId', 'StatusCd', 'EntityStatusCd', 'RelatedPartyId', 'Reason', 'StartTime', 'EndTime', 'AppActionCd', 'CreateUserId', 'CreateUserTime', 'UpdateUserId', 'UpdateUserTime', 'TargetUserId', 'TargetGroupId']

Columns after dropping: ['BusinessInteractionId', 'InstanceId', 'StepId', 'ActionCd', 'ActionUserId', 'StatusCd', 'EntityStatusCd', 'RelatedPartyId', 'StartTime', 'EndTime', 'CreateUserId', 'CreateUserTime', 'TargetUserId', 'TargetGroupId']


,BusinessInteractionId,InstanceId,StepId,ActionCd,ActionUserId,StatusCd,EntityStatusCd,RelatedPartyId,StartTime,EndTime,CreateUserId,CreateUserTime,TargetUserId,TargetGroupId
0,10230767264,10003850078,1.000012e+12,SaveStep,573540,W,NEW,347878,2021-03-19 23:26:18.479,2021-03-19 23:26:18.479,573540,2021-03-19 23:26:18.588,0,0
1,10230767256,10003850078,NaN,WorkflowStarted,573540,W,NEW,573540,2021-03-19 23:26:02.260,2021-03-19 23:26:02.260,573540,2021-03-19 23:26:11.510,0,0
2,10230767260,10003850078,1.000012e+12,EmailSent,573540,W,NEW,347878,2021-03-19 23:26:14.104,2021-03-19 23:26:14.104,573540,2021-03-19 23:26:14.729,0,0
3,10232343851,10003850078,NaN,CancelInstance,4,W,NEW,4,2021-03-24 10:05:48.456,2021-03-24 10:05:48.456,4,2021-03-24 10:05:52.972,0,0
4,10230767273,10003850078,1.000012e+12,SaveStep,573540,W,NEW,347878,2021-03-19 23:26:21.202,2021-03-19 23:26:21.202,573540,2021-03-19 23:26:21.390,0,0


In [7]:
df_BI_cleaned.fillna(0, inplace=True)
df_BI_cleaned.isna().sum

<bound method DataFrame.sum of          BusinessInteractionId  InstanceId  StepId  ActionCd  ActionUserId  \
0                        False       False   False     False         False   
1                        False       False   False     False         False   
2                        False       False   False     False         False   
3                        False       False   False     False         False   
4                        False       False   False     False         False   
...                        ...         ...     ...       ...           ...   
1209086                  False       False   False     False         False   
1209087                  False       False   False     False         False   
1209088                  False       False   False     False         False   
1209089                  False       False   False     False         False   
1209090                  False       False   False     False         False   

         StatusCd  EntityStatusCd  RelatedPartyId  StartTime  EndTime  \
0           False           False           False      False    False   
1           False           False           False      False    False   
2           False           False           False      False    False   
3           False           False           False      False    False   
4           False           False           False      False    False   
...           ...             ...             ...        ...      ...   
1209086     False           False           False      False    False   
1209087     False           False           False      False    False   
1209088     False           False           False      False    False   
1209089     False           False           False      False    False   
1209090     False           False           False      False    False   

         CreateUserId  CreateUserTime  TargetUserId  TargetGroupId  
0               False           False         False          False  
1               False           False         False          False  
2               False           False         False          False  
3               False           False         False          False  
4               False           False         False          False  
...               ...             ...           ...            ...  
1209086         False           False         False          False  
1209087         False           False         False          False  
1209088         False           False         False          False  
1209089         False           False         False          False  
1209090         False           False         False          False  

[1209091 rows x 14 columns]>

# Column Types
Let's check the column types and correct the types of columns if needed.

In [8]:
df_BI_cleaned.dtypes

,0
BusinessInteractionId,int64
InstanceId,int64
StepId,float64
ActionCd,object
ActionUserId,int64
StatusCd,object
EntityStatusCd,object
RelatedPartyId,int64
StartTime,object
EndTime,object


- IDs and Cd's are all categorical and start and end times must be datetime type

In [9]:
datetime_columns = ['StartTime', 'EndTime', 'CreateUserTime']
for col in datetime_columns:
    df_BI_cleaned[col] = pd.to_datetime(df_BI_cleaned[col], errors='coerce')

categorical_columns = ['ActionCd', 'ActionUserId', 'StatusCd', 'EntityStatusCd', 'RelatedPartyId', 'TargetGroupId']
for col in categorical_columns:
    df_BI_cleaned[col] = df_BI_cleaned[col].astype('category')

print("Data types after conversion:")
df_BI_cleaned.dtypes

Data types after conversion:


,0
BusinessInteractionId,int64
InstanceId,int64
StepId,float64
ActionCd,category
ActionUserId,category
StatusCd,category
EntityStatusCd,category
RelatedPartyId,category
StartTime,datetime64[ns]
EndTime,datetime64[ns]


In [10]:
num_distinct_stepid = df_BI_cleaned['StepId'].nunique()
print(f"Number of distinct categories for StepId: {num_distinct_stepid}")

Number of distinct categories for StepId: 229825


## 2. Let's clean the Data Set named 'Steps'

In [11]:
df_S = pd.read_csv("/content/drive/MyDrive/Python Codes/data/raw/Step.csv", sep=';', on_bad_lines='warn')
df_S.head()

,StepId,InstanceId,StepDefCd,StepDefTypeCd,TypeCd,StatusCd,StartTime,ProcessSlotId,Priority,UserId,PoolPartyId,SentUserId,PreviousStepId,ParallelStarterStepId,ParallelBranchId,IsProcessed,TakenTime,ResumeTime,EndTime,EntityStatusCd,Description,CreateUserId,CreateUserTime,UpdateUserId,UpdateUserTime,OrganizationUnitId
0,1000011695142,10003850078,StartStep,Start,Normal,CNC,2021-03-19 23:26:02.260,643,1,573540.0,564906,NaN,NaN,NaN,NaN,True,NaN,NaN,2021-03-24 10:05:52.409,CANCEL,NaN,573540,2021-03-19 23:26:11.229,573540.0,2021-03-19 23:26:21.374,564906.0
1,1000010610388,10003045706,EndStep,End,Normal,C,2021-03-11 09:07:51.803,401,1,280417.0,474605,NaN,1.000010e+12,NaN,NaN,False,NaN,NaN,2021-03-11 09:07:52.522,CANCEL,NaN,280417,2021-03-11 09:07:52.584,280417.0,2021-03-11 09:07:52.616,564906.0
2,1000009934726,10003045706,ProblemManagerStep,UserStep,Normal,C,2021-03-04 16:24:00.645,401,1,280417.0,474605,NaN,1.000010e+12,0.0,NaN,True,2021-03-04 16:24:01.239,NaN,2021-03-11 09:07:51.803,CANCELLATIONAPPROVAL,NaN,206061,2021-03-04 16:24:01.410,280417.0,2021-03-11 09:07:52.366,564906.0
3,1000009932237,10003045706,CancellationApprovalStep,UserStep,Normal,C,2021-03-04 16:05:00.394,401,1,206061.0,347857,NaN,1.000009e+12,0.0,NaN,True,2021-03-04 16:05:00.894,NaN,2021-03-04 16:24:00.645,CANCELLATIONAPPROVAL,NaN,253676,2021-03-04 16:05:01.019,206061.0,2021-03-04 16:24:00.692,471624.0
4,1000009295771,10003045706,L2Step,UserStep,Normal,C,2021-02-26 16:10:40.371,401,1,253676.0,548216,NaN,1.000009e+12,0.0,NaN,True,2021-03-04 14:37:29.014,NaN,2021-03-04 16:05:00.394,OPEN,NaN,277369,2021-02-26 16:10:40.809,253676.0,2021-03-04 16:05:00.487,548216.0


In [12]:
df_S.shape

(229823, 26)

In [13]:
df_S.isnull().sum()

,0
StepId,0
InstanceId,0
StepDefCd,0
StepDefTypeCd,0
TypeCd,0
StatusCd,0
StartTime,0
ProcessSlotId,0
Priority,0
UserId,584


# What to do with NAs?
Let's first drop the columns where we have lots of NAs and also some columns which we will not use.

In [14]:
columns_to_drop = ['SentUserId', 'ParallelBranchId', 'ResumeTime', 'Description', 'ParallelStarterStepId', 'TakenTime', 'UpdateUserId', 'UpdateUserTime']
df_S_cleaned = df_S.drop(columns=columns_to_drop)

print("Original columns:", df_S.columns.tolist())
print("\nColumns after dropping:", df_S_cleaned.columns.tolist())
df_S_cleaned.head()

Original columns: ['StepId', 'InstanceId', 'StepDefCd', 'StepDefTypeCd', 'TypeCd', 'StatusCd', 'StartTime', 'ProcessSlotId', 'Priority', 'UserId', 'PoolPartyId', 'SentUserId', 'PreviousStepId', 'ParallelStarterStepId', 'ParallelBranchId', 'IsProcessed', 'TakenTime', 'ResumeTime', 'EndTime', 'EntityStatusCd', 'Description', 'CreateUserId', 'CreateUserTime', 'UpdateUserId', 'UpdateUserTime', 'OrganizationUnitId']

Columns after dropping: ['StepId', 'InstanceId', 'StepDefCd', 'StepDefTypeCd', 'TypeCd', 'StatusCd', 'StartTime', 'ProcessSlotId', 'Priority', 'UserId', 'PoolPartyId', 'PreviousStepId', 'IsProcessed', 'EndTime', 'EntityStatusCd', 'CreateUserId', 'CreateUserTime', 'OrganizationUnitId']


,StepId,InstanceId,StepDefCd,StepDefTypeCd,TypeCd,StatusCd,StartTime,ProcessSlotId,Priority,UserId,PoolPartyId,PreviousStepId,IsProcessed,EndTime,EntityStatusCd,CreateUserId,CreateUserTime,OrganizationUnitId
0,1000011695142,10003850078,StartStep,Start,Normal,CNC,2021-03-19 23:26:02.260,643,1,573540.0,564906,NaN,True,2021-03-24 10:05:52.409,CANCEL,573540,2021-03-19 23:26:11.229,564906.0
1,1000010610388,10003045706,EndStep,End,Normal,C,2021-03-11 09:07:51.803,401,1,280417.0,474605,1.000010e+12,False,2021-03-11 09:07:52.522,CANCEL,280417,2021-03-11 09:07:52.584,564906.0
2,1000009934726,10003045706,ProblemManagerStep,UserStep,Normal,C,2021-03-04 16:24:00.645,401,1,280417.0,474605,1.000010e+12,True,2021-03-11 09:07:51.803,CANCELLATIONAPPROVAL,206061,2021-03-04 16:24:01.410,564906.0
3,1000009932237,10003045706,CancellationApprovalStep,UserStep,Normal,C,2021-03-04 16:05:00.394,401,1,206061.0,347857,1.000009e+12,True,2021-03-04 16:24:00.645,CANCELLATIONAPPROVAL,253676,2021-03-04 16:05:01.019,471624.0
4,1000009295771,10003045706,L2Step,UserStep,Normal,C,2021-02-26 16:10:40.371,401,1,253676.0,548216,1.000009e+12,True,2021-03-04 16:05:00.394,OPEN,277369,2021-02-26 16:10:40.809,548216.0


In [15]:
df_S_cleaned.isna().sum()

,0
StepId,0
InstanceId,0
StepDefCd,0
StepDefTypeCd,0
TypeCd,0
StatusCd,0
StartTime,0
ProcessSlotId,0
Priority,0
UserId,584


- PreviousStepId is empty for those that have StartStep as the previous step. So we'll replace them with 0 as we did for the previous data set.
- EndTime is Na for those that are currently being processed.
- UserId and OrganizationUnitId, we'll consider them later

In [16]:
df_S_cleaned['PreviousStepId'].fillna(0, inplace=True)
df_S_cleaned.isna().sum()

/tmp/ipython-input-2389435826.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_S_cleaned['PreviousStepId'].fillna(0, inplace=True)


,0
StepId,0
InstanceId,0
StepDefCd,0
StepDefTypeCd,0
TypeCd,0
StatusCd,0
StartTime,0
ProcessSlotId,0
Priority,0
UserId,584


# Column Types
Let's check the column types and correct the types of columns if needed.

In [17]:
df_S_cleaned.dtypes

,0
StepId,int64
InstanceId,int64
StepDefCd,object
StepDefTypeCd,object
TypeCd,object
StatusCd,object
StartTime,object
ProcessSlotId,int64
Priority,int64
UserId,float64


In [18]:
datetime_columns = ['StartTime', 'EndTime', 'CreateUserTime']
for col in datetime_columns:
    df_S_cleaned[col] = pd.to_datetime(df_S_cleaned[col], errors='coerce')

categorical_columns = ['StepDefCd', 'StepDefTypeCd', 'StatusCd', 'TypeCd', 'Priority', 'EntityStatusCd']
for col in categorical_columns:
    df_S_cleaned[col] = df_S_cleaned[col].astype('category')

print("Data types after conversion:")
df_S_cleaned.dtypes

Data types after conversion:


,0
StepId,int64
InstanceId,int64
StepDefCd,category
StepDefTypeCd,category
TypeCd,category
StatusCd,category
StartTime,datetime64[ns]
ProcessSlotId,int64
Priority,category
UserId,float64


# 3. Let's clean the data set named Instances

In [19]:
df_I = pd.read_csv("/content/drive/MyDrive/Python Codes/data/raw/Instance.csv", sep=';', on_bad_lines='warn')
df_I.head()

,InstanceId,ApplicationCd,WfDefinitionId,WfDefinitionCd,StatusCd,InstanceSourceId,InstanceSourceCd,Priority,ProcessSlotId,StartTime,UserId,MasterId,EndTime,SlaTime,EntityStatusCd,Description,RelatedInstanceId,RelatedStepId,RelationType,CreateUserId,CreateUserTime,UpdateUserId,UpdateUserTime,groupname,division,team1,unit,startgroupname,LastStepId
0,10003850078,IctProblem,4095,ICT_Problem,CNC,10004,DynamicService,1,643,2021-03-19 23:26:02.260,573540,0,2021-03-24 10:05:52.409,NaN,CANCEL,ICT Problem Yönetimi,10003850078,0,InstanceRelationType.Master,573540,2021-03-19 23:26:10.807,4,2021-03-24 10:05:52.409,EE-RHB-HRH-RHC,RBINCHB HCBIRECHER & BHNBIEREHB,RBINCHB HCBIRECHER CIHHBRR RRERBBRBEE,HJREHRBI RBINCHB HCBIRECHER,EE-RHB-HRH-RHC,1.000012e+12
1,10003045706,IctProblem,4095,ICT_Problem,CMP,10006,DynamicService,1,401,2021-02-25 16:02:58.628,206061,0,2021-03-11 09:08:16.600,NaN,CANCEL,ICT Problem Yönetimi,10003045706,0,InstanceRelationType.Master,206061,2021-02-25 16:02:59.144,280417,2021-03-11 09:08:17.694,EE-RHB-HRH-RHC,RBINCHB HCBIRECHER & BHNBIEREHB,RBINCHB HCBIRECHER CIHHBRR RRERBBRBEE,HJREHRBI RBINCHB HCBIRECHER,EE-RHB-HRH-RHC,1.000011e+12
2,10002677992,IctProblem,4087,ICT_Problem,CMP,10006,DynamicService,1,86,2021-02-15 21:11:11.857,431803,0,2021-02-25 18:39:42.533,NaN,CANCEL,ICT Problem Yönetimi,10002677992,0,InstanceRelationType.Master,431803,2021-02-15 21:11:12.385,431803,2021-02-15 21:11:21.808,EE-RHB-HRH-RHC,RBINCHB HCBIRECHER & BHNBIEREHB,RBINCHB HCBIRECHER CIHHBRR RRERBBRBEE,HJREHRBI RBINCHB HCBIRECHER,EE-RHB-HRH-RHC,1.000008e+12
3,10005932762,IctProblem,4110,ICT_Problem,CMP,10001,DynamicService,1,371,2021-05-17 21:58:06.871,415599,0,2021-05-18 01:13:38.387,NaN,CANCEL,ICT Problem Yönetimi,10005932762,0,InstanceRelationType.Master,415599,2021-05-17 21:58:07.199,316614,2021-05-18 01:13:40.309,EE-RHB-RHH-CHE,RBINCHB HCBIRECHER & BHNBIEREHB,CHE HCBIRECHER,RBINCHB HCBIRECHER HBEEBI,EE-RHB-RHH-CHE,1.000018e+12
4,10005828208,IctProblem,4110,ICT_Problem,CNC,10001,DynamicService,1,593,2021-05-14 00:20:52.924,415579,0,2021-05-17 04:00:34.401,NaN,CANCEL,ICT Problem Yönetimi,10005828208,0,InstanceRelationType.Master,415579,2021-05-14 00:20:53.128,4,2021-05-17 04:00:34.401,EE-RHB-RHH-CHE,RBINCHB HCBIRECHER & BHNBIEREHB,CHE HCBIRECHER,RBINCHB HCBIRECHER HBEEBI,EE-RHB-RHH-CHE,1.000018e+12


In [20]:
print(f"Number of rows in df_I: {df_I.shape[0]}")
print(f"Number of columns in df_I: {df_I.shape[1]}")

Number of rows in df_I: 35051
Number of columns in df_I: 29


In [21]:
df_I.isnull().sum()

,0
InstanceId,0
ApplicationCd,0
WfDefinitionId,0
WfDefinitionCd,0
StatusCd,0
InstanceSourceId,0
InstanceSourceCd,7698
Priority,0
ProcessSlotId,0
StartTime,0


# What to do with NAs?
- Let's first drop the columns where we have lots of NAs. In this case, we'll drop SlaTime. We keep InstanceSourceCd for now.
- Groupname, team1 and unit are going to be important for us. Since the number of NAs are relatively small, we'll also drop the rows where we have NAs for these columns.

In [22]:
columns_to_drop = ['SlaTime', 'UserId', 'RelatedInstanceId', 'RelationType', 'CreateUserId', 'UpdateUserId']
df_I_cleaned = df_I.drop(columns=columns_to_drop)

columns_NA = ['groupname', 'team1', 'unit', 'LastStepId']
df_I_cleaned.dropna(subset=columns_NA, inplace=True)

print(f"Shape of df_I_cleaned after dropping 'SlaTime' and NA 'groupname, team1, unit' rows: {df_I_cleaned.shape}")
print("\nNull values after operations:")
df_I_cleaned.isnull().sum()

Shape of df_I_cleaned after dropping 'SlaTime' and NA 'groupname, team1, unit' rows: (34249, 23)

Null values after operations:


,0
InstanceId,0
ApplicationCd,0
WfDefinitionId,0
WfDefinitionCd,0
StatusCd,0
InstanceSourceId,0
InstanceSourceCd,7280
Priority,0
ProcessSlotId,0
StartTime,0


# Column Types
Let's check the column types and correct the types of columns if needed.

In [23]:
df_I_cleaned.dtypes

,0
InstanceId,int64
ApplicationCd,object
WfDefinitionId,int64
WfDefinitionCd,object
StatusCd,object
InstanceSourceId,int64
InstanceSourceCd,object
Priority,int64
ProcessSlotId,int64
StartTime,object


In [24]:
datetime_columns = ['StartTime', 'EndTime', 'CreateUserTime', 'UpdateUserTime']
for col in datetime_columns:
    df_I_cleaned[col] = pd.to_datetime(df_I_cleaned[col], errors='coerce')

categorical_columns = ['ApplicationCd', 'WfDefinitionCd', 'StatusCd', 'InstanceSourceCd', 'Priority', 'EntityStatusCd', 'Description', 'groupname', 'division', 'team1', 'unit', 'startgroupname']
for col in categorical_columns:
    df_I_cleaned[col] = df_I_cleaned[col].astype('category')

print("Data types after conversion:")
df_I_cleaned.dtypes

Data types after conversion:


,0
InstanceId,int64
ApplicationCd,category
WfDefinitionId,int64
WfDefinitionCd,category
StatusCd,category
InstanceSourceId,int64
InstanceSourceCd,category
Priority,category
ProcessSlotId,int64
StartTime,datetime64[ns]


### Export to CSV

In [26]:
# Export to CSV
# index=False prevents writing the DataFrame index as a column in the CSV
df_BI_cleaned.to_csv('df_BI_cleaned.csv', index=False)
print("df_BI_cleaned exported to 'df_BI_cleaned.csv'")
df_S_cleaned.to_csv('df_S_cleaned.csv', index=False)
print("df_S_cleaned exported to 'df_S_cleaned.csv'")
df_I_cleaned.to_csv('df_I_cleaned.csv', index=False)
print("df_I_cleaned exported to 'df_I_cleaned.csv'")

df_BI_cleaned exported to 'df_BI_cleaned.csv'
df_S_cleaned exported to 'df_S_cleaned.csv'
df_I_cleaned exported to 'df_I_cleaned.csv'


### Export to Excel

In [ ]:
# Export to Excel
# sheet_name specifies the name of the sheet in the Excel file
df_BI_cleaned.to_excel('df_BI_cleaned.xlsx', sheet_name='Sheet1', index=False)
print("df_BI_cleaned exported to 'df_BI_cleaned.xlsx'")

### Export to Parquet

In [ ]:
# Export to Parquet
# Parquet is a columnar storage format, good for performance on large datasets
df_BI_cleaned.to_parquet('df_BI_cleaned.parquet', index=False)
print("df_BI_cleaned exported to 'df_BI_cleaned.parquet'")